In [1]:
import numpy as np
import scipy as sp
import gymnasium as gym
from gymnasium.spaces import Box, Discrete, multi_binary
import matplotlib

In [2]:
# Option Pricers

def BSprice(S: list[float], k0: np.ndarray, k1: np.ndarray, r: float, T: float, sigma: float) -> list[np.ndarray]:

    d1 = ( np.log(k0/S[0]) + 0.5*sigma*sigma*np.sqrt(T) ) / sigma*np.sqrt(T)
    d2 = d1 - sigma*np.sqrt(T)
    C0 = sp.norm.cdf(d1)*S[0]-sp.norm.cdf(d2)*k0*np.exp(-r*T)
    P0 = C0 - S[0] + k0*np.exp(-r*T)

    d1 = ( np.log(k1/S[1]) + 0.5*sigma*sigma*np.sqrt(T) ) / sigma*np.sqrt(T)
    d2 = d1 - sigma*np.sqrt(T)
    C1 = sp.norm.cdf(d1)*S[1]-sp.norm.cdf(d2)*k1*np.exp(-r*T) 
    P1 = C1 - S[1] + k1*np.exp(-r*T)

    return [C0, P0, C1, P1]

def BGprice(S: list[float], k0: np.ndarray, k1: np.ndarray, r: float, T: float, \
            alpha: float, lam: float, eta: float, N: int, \
             bp: float, cp: float, bn: float, cn: float) -> list[np.ndarray]:
    beta = np.log(S[0])-lam*N/2
    k = beta+(np.cumsum(np.ones(N,1))-1)*lam
    u = (np.cumsum(np.ones(N,1))-1)*eta
    w = np.ones(N,1)*eta
    w[0] = w[0]/2
    x = np.exp(-1j*beta*u)*Psi_BG(u,bp,cp,bn,cn,T,r,alpha,S[0])*w
    Call = np.real((np.exp(-alpha*k)/np.pi)*sp.fft(x,N))
    kk = np.log(k0)
    C0 = np.interp(k,Call,kk)
    P0 = C0 - S[0] + k0*np.exp(-r*T)

    beta = np.log(S[1])-lam*N/2
    k = beta+(np.cumsum(np.ones(N,1))-1)*lam
    u = (np.cumsum(np.ones(N,1))-1)*eta
    w = np.ones(N,1)*eta
    w[0] = w[0]/2
    x = np.exp(-1j*beta*u)*Psi_BG(u,bp,cp,bn,cn,T,r,alpha,S[0])*w
    Call = np.real((np.exp(-alpha*k)/np.pi)*sp.fft(x,N))
    kk = np.log(k1)
    C1 = np.interp(k,Call,kk)
    P1 = C1 - S[1] + k1*np.exp(-r*T)

    return [C0, P0, C1, P1]

def Phi_BG(u: float, r: float, k: np.ndarray, T: float, bp: float, cp: float, bn: float, cn: float) -> list[float]:
    d0 = 1j*u*(np.log(k)+T*(r+cp*np.log(1-bp)+cn*np.log(1+bn)))
    Phi = np.exp(d0)*((1-1j*u*bp)^(-T*cp))*((1+1j*u*bn)^(-T*cn))
    return Phi

def Psi_BG(u: float, r: float, k: np.ndarray, T: float, alpha: float, bp: float, cp: float, bn: float, cn: float) -> list[float]: 
    Psi = (np.exp(-r*T)/((alpha+1j*u)/(alpha+1j*u+1)))/Phi_BG(u-(alpha+1)*1j,bp,cp,bn,cn,T,k,r)
    return Psi



In [3]:
class BenchmarkReplication(gym.Env):

    def __init__(self, W: float, N: int, Dynamics: str, start_time = 0, T: float = 26, dT: float = 1, r: float = 0, *kwargs):
        
        self.Dynamics = Dynamics
        self.sigma = kwargs.get('sigma',None)
        self.mu = kwargs.get('mu',None)
        self.bp = kwargs.get('bp',None)
        self.cp = kwargs.get('cp',None)
        self.bn = kwargs.get('bn',None)
        self.cn = kwargs.get('cn',None)

        S = [500,25]
        kmin = 0.7*S 
        kmax = 1.3*S
        k0 = np.linspace(kmin[0],kmax[0],N/2)
        k1 = np.linspace(kmin[1],kmax[0],N/2)

        O = BSprice(S,k0,k1,r,dT,self.sigma)

        self.start_time = start_time
        self.T = T
        self.dT = dT
        
        self.W0 = W # Initial Wealth
        self.W = W # Current Wealth
        self.N = N # There are N calls and N put options for each underlying. Then the player needs to make 4N-1 decisions.
        
        self.action_spaces = Box(low = -np.inf, high = np.inf, shape = (4*N-1,))
        self.observation_space = Box(low = -np.inf, high = np.inf, shape = (4*N+2,)), # prices of 4N options and 2 underlying assets 
        self.p = np.zeros(4*N) # current position in each option
        
        
        
    def step(self, action):
        T = self.T
        N = self.N
        p = self.p
      
        if self.time < T-1: # If it is the final moment, we liquidate all the positions
            self.time += 1
            S = self.time_series[self.time]
            kmin = 0.7*S 
            kmax = 1.3*S
            k0 = np.linspace(kmin[0],kmax[0],N/2)
            k1 = np.linspace(kmin[1],kmax[0],N/2)
            payoff = 0
            for n in range(N/2):
                payoff = payoff + action[0][n]*max(S[0] - k0[n],0) + action[1][n]*max(S[1] - k1[n],0)
            payoff = payoff + action[0][N/2-1]*max(S[0] - k0[N/2-1],0) + (self.W-sum(action))*max(S[1] - k1[n],0)
            
            self.W[1] += payoff + self.cash - S[0]*S[0]/sum(S) - S[1]*S[1]/sum(S) #The benchmark formed by SPY and XLE is subtracted
            return (payoff, self.time, S)
        else:
            raise IndexError("The game is over! Please reset the game.")

            # we return cash, position,
        
        
    def reset(self):
        T = self.T
        N = self.N
        self.p = np.zeros(N)
        self.W = self.W0 # the initial cash
        self.time = self.start_time # the current time is zero
        
        
        eps0 = [self.sigma[0]*np.random.rand(1) for i in range(T)]
        eps1 = [self.sigma[1]*np.random.rand(1) for i in range(T)]
        
        self.time_series = [[sum(eps0[:i]), sum(eps1[:i])] for i in range(T)]

        return (self.cash,self.time,self.time_series[:self.time+1])

    def render(self):
        pass

This should serve as basic model to construct a portfolio of options to replicate the SPY/XLE benchmark.
Some things to do inclulde:
- playing a game (e.g., def a ``play`` function that first reset the game and subsequently runs step until game ends)  (Note: there are surely errors in the code above as I have not tested)
- adding correlation in the generation of the time series
- run the game over time series of market data
- determine optimal length of each time step (i.e. optimal maturities to trade)